## Exercise 2 - Question 1

### Import packages

In [1]:
import numpy as np
import pandas as pd
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.vector_ar.var_model import VAR
from statsmodels.tsa.ar_model import ar_select_order
import torch.nn as nn
import torch
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from sklearn.preprocessing import StandardScaler


### Load Data and Preparations

In [2]:
# load dataset
filepath = 'https://raw.githubusercontent.com/selva86/datasets/master/Raotbl6.csv'
df = pd.read_csv(filepath, parse_dates=['date'], index_col='date')

# test for causality between two time series - RGNP & GNP
test = 'ssr_chi2test'
test_result = grangercausalitytests(df[['rgnp','pgnp']], maxlag=10, verbose=False)
p_values = [round(test_result[i+1][0][test][1],4) for i in range(10)]
print('p_values = ',p_values)
print('min_p_value = ',np.min(p_values))

# test for stationarity
r = adfuller(df['rgnp'], autolag='AIC')
output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
p_value = output['pvalue']
print(output)

# difference the series to get stationarity
v = df.rgnp.values
dv = np.diff(v)
r = adfuller(dv, autolag='AIC')
output = {'test_statistic':round(r[0], 4), 'pvalue':round(r[1], 4), 'n_lags':round(r[2], 4), 'n_obs':r[3]}
p_value = output['pvalue']
print(output)

# split train & test
train = dv[:-10]
test = dv[-10:]

p_values =  [0.3256, 0.1849, 0.111, 0.2159, 0.2772, 0.0106, 0.0152, 0.0126, 0.0211, 0.0401]
min_p_value =  0.0106
{'test_statistic': 0.6419, 'pvalue': 0.9886, 'n_lags': 2, 'n_obs': 120}
{'test_statistic': -5.4276, 'pvalue': 0.0, 'n_lags': 1, 'n_obs': 120}


### 2-A - AR

In [3]:
# A - predict last 10 observations of rgnp with AR
sel = ar_select_order(train, 5, ic='aic')
res = sel.model.fit()
print(res.summary())
print(res.model.predict(params=res.params, start=len(train), end=len(train)+len(test)-1, dynamic=False))

                            AutoReg Model Results                             
Dep. Variable:                      y   No. Observations:                  112
Model:                     AutoReg(2)   Log Likelihood                -516.599
Method:               Conditional MLE   S.D. of innovations             26.507
Date:                Thu, 23 Dec 2021   AIC                           1041.199
Time:                        12:27:36   BIC                           1052.001
Sample:                             2   HQIC                          1045.580
                                  112                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const         12.3223      3.339      3.691      0.000       5.779      18.866
y.L1           0.2304      0.094      2.444      0.015       0.046       0.415
y.L2           0.1496      0.094      1.588      0.1

### 2-B VAR

In [4]:
# B - predict last 10 observations of rgnp & pgnp with VAR
v = df.loc[:, ['rgnp', 'pgnp']].values
dv = np.diff(v, axis=0)
train = dv[:-10]
test = dv[-10:]

model = VAR(train)
results = model.fit(maxlags=5, ic='aic')
print(results.summary())
lag_order = results.k_ar
print(results.forecast(train, 5))

  Summary of Regression Results   
Model:                         VAR
Method:                        OLS
Date:           Thu, 23, Dec, 2021
Time:                     12:27:44
--------------------------------------------------------------------
No. of Equations:         2.00000    BIC:                    5.60700
Nobs:                     111.000    HQIC:                   5.51995
Log likelihood:          -612.064    FPE:                    235.230
AIC:                      5.46054    Det(Omega_mle):         223.013
--------------------------------------------------------------------
Results for equation y1
           coefficient       std. error           t-stat            prob
------------------------------------------------------------------------
const         8.111089        19.043517            0.426           0.670
L1.y1         0.262861         0.093694            2.806           0.005
L1.y2         0.312022         0.957212            0.326           0.744

Results for equation 

### 2-C -LSTM

In [5]:
# C - predict last 10 observations with LSTM
class lstm(nn.Module):
    def __init__(self, input_size, hidden_size):
        super().__init__()
        hidden_state = torch.randn(1, 1, hidden_size)
        cell_state = torch.randn(1, 1, hidden_size)
        self.hidden = (hidden_state, cell_state)
        self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size, batch_first=True)
        self.lin = nn.Linear(hidden_size, 1)
    def forward(self, input):
        lstm_out, _ = self.lstm(input)
        x = self.lin(lstm_out[:, -1, :])
        return x

class GDPDataset(Dataset):
    def __init__(self, data, window_size=5):
        self.train_data = []
        self.train_labels = []
        for i in range(len(data) - window_size):
            self.train_data.append(data[i:i+window_size])
            self.train_labels.append(data[i+window_size, 1])
        self.train_data = torch.Tensor(self.train_data)
        self.train_labels = torch.Tensor(self.train_labels)
    def __len__(self):
        return len(self.train_labels)

    def __getitem__(self, idx):
        return self.train_data[idx], self.train_labels[idx]

def train_loop(dataloader, model, loss_fn, optimizer):
    for X, y in dataloader:
        # Compute prediction and loss
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

def test_loop(dataloader, model, loss_fn):
    num_batches = len(dataloader)
    test_loss = 0

    with torch.no_grad():
        for X, y in dataloader:
            pred = model(X)
            test_loss += loss_fn(pred, y).item()

    test_loss /= num_batches
    print(f"Test Error: \n MSE: {(test_loss):>0.1f}%")
    return test_loss

# use all data to predict the rgnp - assume window_size = 5
v = df.values
dv = np.diff(v, axis=0)
ss = StandardScaler(copy=False)
dv = ss.fit_transform(dv)
train = dv[:-10]
test = dv[-10:]

# hyper parameters
learning_rate = 1e-2
batch_size = 64
epochs = 10

# build dataset

trainDatast = GDPDataset(train)
testDatast = GDPDataset(test)

# data loaders
train_dataloader = DataLoader(trainDatast, batch_size=16, shuffle=True)
test_dataloader = DataLoader(testDatast, batch_size=16, shuffle=True)


loss = nn.MSELoss()
hidden_sizes = [4, 8, 16, 32, 64, 128, 256]
res = {'hidden_size': [], 'test_loss': []}
for h in hidden_sizes:
    model = lstm(input_size=8, hidden_size=h)
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    for t in range(epochs):
        print(f"Epoch {t + 1}\n-------------------------------")
        train_loop(train_dataloader, model, loss, optimizer)
        test_loss = test_loop(test_dataloader, model, loss)
    res['test_loss'].append(test_loss)
    res['hidden_size'].append(h)
res = pd.DataFrame(res)
print(f"best hidden size of lstm with window size=5: {res[res['test_loss']==res['test_loss'].min()]['hidden_size'].values[0]}")

######################################
#        best hidden size = 128
######################################

C:\Users\jvjos\anaconda3\envs\ex_2\lib\site-packages\ipykernel_launcher.py:22: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  ..\torch\csrc\utils\tensor_new.cpp:201.)
C:\Users\jvjos\anaconda3\envs\ex_2\lib\site-packages\torch\nn\modules\loss.py:520: UserWarning: Using a target size (torch.Size([16])) that is different to the input size (torch.Size([16, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
C:\Users\jvjos\anaconda3\envs\ex_2\lib\site-packages\torch\nn\modules\loss.py:520: UserWarning: Using a target size (torch.Size([11])) that is different to the input size (torch.Size([11, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  retur

Epoch 1
-------------------------------
Test Error: 
 MSE: 2.3%
Epoch 2
-------------------------------
Test Error: 
 MSE: 2.0%
Epoch 3
-------------------------------
Test Error: 
 MSE: 1.7%
Epoch 4
-------------------------------
Test Error: 
 MSE: 1.5%
Epoch 5
-------------------------------
Test Error: 
 MSE: 1.5%
Epoch 6
-------------------------------
Test Error: 
 MSE: 1.7%
Epoch 7
-------------------------------
Test Error: 
 MSE: 1.8%
Epoch 8
-------------------------------
Test Error: 
 MSE: 1.8%
Epoch 9
-------------------------------
Test Error: 
 MSE: 1.8%
Epoch 10
-------------------------------
Test Error: 
 MSE: 1.7%
Epoch 1
-------------------------------
Test Error: 
 MSE: 1.7%
Epoch 2
-------------------------------
Test Error: 
 MSE: 1.7%
Epoch 3
-------------------------------
Test Error: 
 MSE: 1.8%
Epoch 4
-------------------------------
Test Error: 
 MSE: 2.0%
Epoch 5
-------------------------------
Test Error: 
 MSE: 2.0%
Epoch 6
-------------------------------